##### Copyright 2024 Google LLC.

In [107]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Question Answering using Gemini, LangChain, and Chroma

## Overview

[Gemini](https://ai.google.dev/models/gemini) is a family of generative AI models that lets developers generate content and solve problems. These models are designed and trained to handle both text and images as input.

[LangChain](https://www.langchain.com/) is a data framework designed to make integration of Large Language Models (LLM) like Gemini easier for applications.

[Chroma](https://docs.trychroma.com/) is an open-source embedding database focused on simplicity and developer productivity. Chroma allows users to store embeddings and their metadata, embed documents and queries, and search the embeddings quickly.

In this notebook, you'll learn how to create an application that answers questions using data from a website with the help of Gemini, LangChain, and Chroma.

## Setup

 - Modified to factor all the setup on the requirements.txt
 - Also utilize build an app and scripts to allow easier showcase and code management.


### Web Agent Loading from Website

In [108]:
import os
import pprint
import re
import markdown

In [109]:
# api key setup
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
youtube_api_key = os.environ.get("youtube_api_key")
open_ai_key = os.environ.get("open_ai_key")
google_api_key = os.environ.get("google_ai_studio_key")

In [110]:
import os

def list_files(path, verbose=False)->list:
    """Lists files at the specified path, including those in subdirectories."""

    _list = []
    for root, directories, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            _list.append(file_path)
            if  verbose:
                print(file_path)
    return _list
# Example usage:
# path_to_list = "/your/designated/path"  # Replace with the actual path


In [111]:
from langchain_community.document_loaders import PyPDFLoader

def pdf_loader_doc(pdf_path_list)->dict:
    pdf_doc_dict={}
    for i in pdf_path_list:

        # pdf loader
        loader = PyPDFLoader(i)
        pages = loader.load_and_split()
        
        # filename
        path_components = i.split("/")
        filename = path_components[-1]

        pdf_doc_dict[filename] = pages
    return pdf_doc_dict

In [112]:
import re

def extract_markdown_links(path, verbose=False)->dict:
    # Markdown (using the markdown library)
    with open(path, "r") as file:
        text = file.read()

    link_pattern = r"\[(.*?)\]\((.*?)\)"  # Matches link text and URL within parentheses
    links = re.findall(link_pattern, text)

    # text
    links_dict = {}
    for link_name, url in links:
        links_dict[link_name] = url
        if verbose:
            print(f"Link Name: {link_name}")
            print(f"URL: {url}")
    return links_dict

In [113]:
from langchain_community.document_loaders import WebBaseLoader

def web_loader_doc(web_path_list)->dict:
    web_doc_dict={}
    key_list = [key for key,val in web_path_list.items()]
    val_list = [val for key,val in web_path_list.items()]
    loader = WebBaseLoader(val_list)
    docs = loader.load()
    # Extract the text from the website data document
    # text_content = docs[0].page_content
    for i in range(len(docs)):
        web_doc_dict[key_list[i]] = docs[i].page_content
    return web_doc_dict

In [114]:
import re
def extract_youtube_ids(url, verbose=False):

    # Extract the video ID with a regular expression
    match = re.search(r"v=([^&]+)", url)

    if match:
        video_id = match.group(1)
        if verbose:
            print(f"Video ID: {video_id}")
    else:
        if verbose:
            print("Error: Could not extract video ID from URL.")
    return video_id

In [115]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import HTMLHeaderTextSplitter
def web_html_loader_doc(web_path_list)-> dict:
    key_list = [key for key,val in web_path_list.items()]
    val_list = [val for key,val in web_path_list.items()]
    web_doc_dict={}
    for idx, url in enumerate(val_list):
        # url = "https://plato.stanford.edu/entries/goedel/"

        headers_to_split_on = [
            ("h1", "Header 1"),
            ("h2", "Header 2"),
            # ("h3", "Header 3"),
            # ("h4", "Header 4"),
        ]

        html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

        # for local file use html_splitter.split_text_from_file(<path_to_file>)
        html_header_splits = html_splitter.split_text_from_url(url)

        chunk_size = 1000
        chunk_overlap = 50
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )

        # Split
        splits = text_splitter.split_documents(html_header_splits)
        web_doc_dict[key_list[idx]] = splits
    return web_doc_dict

In [116]:
import json
import youtube_transcript_api
from googleapiclient.discovery import build

def youtube_get_transcript(video_id:list, api_key:str)->list:
    """Retrieves the transcript of a YouTube video."""
    youtube = build('youtube', 'v3', developerKey=youtube_api_key)
    

    transcript_list = []
    for i in video_id:
        transcript = youtube_transcript_api.YouTubeTranscriptApi.get_transcript(i)
        transcript_text = []
        for segment in transcript:
            transcript_text.append(segment['text'])

        joined_text = " ".join(transcript_text)
    transcript_list.append(joined_text)
        
    return transcript_list

In [117]:
def youtube_get_title(video_id:list, api_key:str)->list:
    """Retrieves the title of a YouTube video."""
    youtube = build('youtube', 'v3', developerKey=youtube_api_key)
    # Get video information
    video_title_list = []
    for id in video_id:
        request = youtube.videos().list(
            part="snippet",
            id=id
        )
        response = request.execute()

        # Extract video title
        video_title = response['items'][0]['snippet']['title']
        video_title_list.append(video_title)
    return video_title_list

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def youtube_transcript_sciprt(title_list, youtube_transcripts_list):
    doc_dict = {}
    for title, transcripts in zip(title_list, youtube_transcripts_list):
        text_splitter = RecursiveCharacterTextSplitter(
            # Set a really small chunk size, just to show.
            chunk_size=1000,
            chunk_overlap=50,
            length_function=len,
            is_separator_regex=False,
        )

        texts = text_splitter.create_documents([transcripts])
        doc_dict[title]=texts
    return doc_dict

# def combined_title_transcripts_list(title_list, transcripts_list):
#     return [Document(page_content=y, metadata={"source": "youtube", "title":x}) for x,y in zip(title_list, transcripts_list)]

## PDF

In [118]:
claims_pdf_files = list_files('../data/pdf_raw/claims')
claims_pdf_doc = pdf_loader_doc(claims_pdf_files)

In [119]:
# add meta data
for i in claims_pdf_doc:
    for idx, doc in enumerate(claims_pdf_doc[i]):
        claims_pdf_doc[i][idx].metadata = {"Source":"pdf", "functions":"claims"}


In [120]:
disease_pdf_files = list_files('../data/pdf_raw/disease')
disease_pdf_doc = pdf_loader_doc(disease_pdf_files)

In [121]:
# add meta data
for i in disease_pdf_doc:
    for idx, doc in enumerate(disease_pdf_doc[i]):
        disease_pdf_doc[i][idx].metadata = {"Source":"pdf", "functions":"disease"}

## Web

In [122]:
disease_web_dict = extract_markdown_links("../data/web/disease_links.md")
insurance_web_dict = extract_markdown_links("../data/web/insurance_links.md")
disease_youtube_dict = extract_markdown_links("../data/web/youtube_disease_links.md")

In [123]:
disease_youtube_dict = {key:extract_youtube_ids(val) for key,val in disease_youtube_dict.items()}

In [124]:
disease_web_doc = web_html_loader_doc(disease_web_dict)
insurance_web_doc = web_html_loader_doc(insurance_web_dict)

In [137]:
# investigation

In [135]:
insurance_web_dict = extract_markdown_links("../data/web/insurance_links.md")
insurance_web_doc = web_html_loader_doc(insurance_web_dict)

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'
}

session = requests.Session()
response = session.get('https://www.example.com/', headers=headers)

In [136]:
insurance_web_doc['FWD CI Plus Claim FAQ']

[Document(page_content='Enable JavaScript and cookies to continue')]

In [20]:
# add meta data
for i in disease_web_doc:
    for idx, doc in enumerate(disease_web_doc[i]):
        disease_web_doc[i][idx].metadata = {"Source":"web", "functions":"disease"}

In [21]:
# add meta data
for i in insurance_web_doc:
    for idx, doc in enumerate(insurance_web_doc[i]):
        insurance_web_doc[i][idx].metadata = {"Source":"web", "functions":"claims"}

## Youtube

In [22]:
video_ids=[]
for name, video_id in disease_youtube_dict.items():
    video_ids.append(video_id)
youtube_title_list = youtube_get_title(video_ids, youtube_api_key)
youtube_transcripts_list = youtube_get_transcript(video_ids, youtube_api_key)
disease_youtube_doc = youtube_transcript_sciprt(youtube_title_list, youtube_transcripts_list)

## Basic steps
LLMs are trained offline on a large corpus of public data. Hence they cannot answer questions based on custom or private data accurately without additional context.

If you want to make use of LLMs to answer questions based on private data, you have to provide the relevant documents as context alongside your prompt. This approach is called Retrieval Augmented Generation (RAG).

You will use this approach to create a question-answering assistant using the Gemini text model integrated through LangChain. The assistant is expected to answer questions about AI and climate change. To make this possible you will add more context to the assistant using data from a website.

In this tutorial, you'll implement the two main components in an RAG-based architecture:

1. Retriever

    Based on the user's query, the retriever retrieves relevant snippets that add context from the document. In this tutorial, the document is the website data.
    The relevant snippets are passed as context to the next stage - "Generator".

2. Generator

    The relevant snippets from the website data are passed to the LLM along with the user's query to generate accurate answers.

You'll learn more about these stages in the upcoming sections while implementing the application.

## Import the required libraries

In [23]:
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma

## Retriever

In this stage, you will perform the following steps:

1. Read and parse the website data using LangChain.

2. Create embeddings of the website data.

    Embeddings are numerical representations (vectors) of text. Hence, text with similar meaning will have similar embedding vectors. You'll make use of Gemini's embedding model to create the embedding vectors of the website data.

3. Store the embeddings in Chroma's vector store.
    
    Chroma is a vector database. The Chroma vector store helps in the efficient retrieval of similar vectors. Thus, for adding context to the prompt for the LLM, relevant embeddings of the text matching the user's question can be retrieved easily using Chroma.

4. Create a Retriever from the Chroma vector store.

    The retriever will be used to pass relevant website embeddings to the LLM along with user queries.

### Read and parse the website data

LangChain provides a wide variety of document loaders. To read the website data as a document, you will use the `WebBaseLoader` from LangChain.

To know more about how to read and parse input data from different sources using the document loaders of LangChain, read LangChain's [document loaders guide](https://python.langchain.com/docs/integrations/document_loaders).

If you only want to select a specific portion of the website data to add context to the prompt, you can use regex, text slicing, or text splitting.

In this example, you'll use Python's `split()` function to extract the required portion of the text. The extracted text should be converted back to LangChain's `Document` format.

In [97]:
def extract_dict_values(_dict):
    return [val for key,val in _dict.items()]

In [98]:
disease_youtube_docs = extract_dict_values(disease_youtube_doc)
disease_web_docs = extract_dict_values(disease_web_doc)
insurance_web_docs = extract_dict_values(insurance_web_doc)
claims_pdf_docs = extract_dict_values(claims_pdf_doc)
disease_pdf_docs = extract_dict_values(disease_pdf_doc)

In [99]:
docs = []
for i in zip(disease_youtube_docs, claims_pdf_docs, disease_pdf_docs, disease_web_docs, insurance_web_docs): #, #
    docs.extend(i)

def flatten_list(nested_list):
  """Flattens a nested list into a single list.

  Args:
    nested_list: A list that may contain other lists.

  Returns:
    A single list containing all elements from the nested list.
  """

  flat_list = []
  for item in nested_list:
    if isinstance(item, list):
      flat_list.extend(flatten_list(item))
    else:
      flat_list.append(item)
  return flat_list

In [100]:
docs = flatten_list(docs)

In [101]:
def should_keep_document(document, min_length=150, min_density=0.2):
  """Checks if a document meets the criteria for being kept.

  Args:
    document: The document to check.
    min_length: The minimum length (in characters) for a document to be kept.
    min_density: The minimum information density (ratio of unique words to total words) for a document to be kept.

  Returns:
    True if the document meets the criteria, False otherwise.
  """

  if len(document) < min_length:
    return False

  word_count = len(document.split())
  unique_words = len(set(document.split()))
  density = unique_words / word_count

  return density >= min_density

In [103]:
docs = [i for i in docs if should_keep_document(i.page_content)]

### Initialize Gemini's embedding model

To create the embeddings from the website data, you'll use Gemini's embedding model, **embedding-001** which supports creating text embeddings.

To use this embedding model, you have to import `GoogleGenerativeAIEmbeddings` from LangChain. To know more about the embedding model, read Google AI's [language documentation](https://ai.google.dev/models/gemini).

In [32]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# If there is no environment variable set for the API key, you can pass the API
# key to the parameter `google_api_key` of the `GoogleGenerativeAIEmbeddings`
# function: `google_api_key = "key"`.

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Store the data using Chroma

To create a Chroma vector database from the website data, you will use the `from_documents` function of `Chroma`. Under the hood, this function creates embeddings from the documents created by the document loader of LangChain using any specified embedding model and stores them in a Chroma vector database.  

You have to specify the `docs` you created from the website data using LangChain's `WebBasedLoader` and the `gemini_embeddings` as the embedding model when invoking the `from_documents` function to create the vector database from the website data. You can also specify a directory in the `persist_directory` argument to store the vector store on the disk. If you don't specify a directory, the data will be ephemeral in-memory.


In [33]:
# Save to disk
vectorstore = Chroma.from_documents(
                     documents=docs,                 # Data
                     embedding=gemini_embeddings,    # Embedding model
                    #  persist_directory="./chroma_db" # Directory to save data
                     )

### Create a retriever using Chroma

You'll now create a retriever that can retrieve website data embeddings from the newly created Chroma vector store. This retriever can be later used to pass embeddings that provide more context to the LLM for answering user's queries.


To load the vector store that you previously stored in the disk, you can specify the name of the directory that contains the vector store in `persist_directory` and the embedding model in the `embedding_function` arguments of Chroma's initializer.

You can then invoke the `as_retriever` function of `Chroma` on the vector store to create a retriever.

In [59]:
# Load from disk
vectorstore_disk = Chroma(
                    #     persist_directory="./chroma_db",       # Directory of db
                        embedding_function=gemini_embeddings   # Embedding model
                   )
# Get the Retriever interface for the store to use later.
# When an unstructured query is given to a retriever it will return documents.
# Read more about retrievers in the following link.
# https://python.langchain.com/docs/modules/data_connection/retrievers/
#
# Since only 1 document is stored in the Chroma vector store, search_kwargs `k`
# is set to 1 to decrease the `k` value of chroma's similarity search from 4 to
# 1. If you don't pass this value, you will get a warning.
retriever = vectorstore_disk.as_retriever( search_kwargs={"k": 15}) #" "score_threshold": .3 search_type ='similarity_score_threshold',

# Check if the retriever is working by trying to fetch the relevant docs related
# to the word 'climate'. If the length is greater than zero, it means that
# the retriever is functioning well.
print(len(retriever.get_relevant_documents("claims for critical illness")))

15


In [60]:
retriever.get_relevant_documents("cardiac arrest")

[Document(page_content='Life after cardiac arrest\nInformation for survivors and their loved ones \nPublished in 2021 by the Swedish Resuscitation Council and the Swedish Heart and Lung Association', metadata={'Source': 'pdf', 'functions': 'disease'}),
 Document(page_content='× SUBSCRIBE TO HEALTHXCHANGE  \nSubscribe to our mailing list to get the updates to your inbox  \nPlease enter a valid email.\xa0Entered email already exists.Please enter a valid email.Subscribe  \nAbout Us  \nCorporate Profile Regional Health System Global Health Newsroom Procurement Contact Us  \nPatient Care  \nConditions & Treatments Specialties & Services Find A Doctor Patient & Visitor Info e-Services  \nResearch  \nResearch & Innovation  \nEducation  \nEducation & Training  \nCareers  \nWhy Choose Us Career Choices Scholarships Sponsorships Job Opportunities Contact Us  \nGiving  \nWhy Give? Get Involved Your Gift at Work Be Informed Make a Gift  \n10 Hospital Boulevard, #19-01 SingHealth Tower. Singapore 1

In [104]:
retriever.get_relevant_documents("provide an summary on cardiac arrest")

[Document(page_content='Life after cardiac arrest\nInformation for survivors and their loved ones \nPublished in 2021 by the Swedish Resuscitation Council and the Swedish Heart and Lung Association', metadata={'Source': 'pdf', 'functions': 'disease'}),
 Document(page_content='7\nEarly defibrillation \n– to restart the heart\nDefbrillation means that an electrical shock is deliv -\nered to the heart using a defibrillator and electrodes \nthat are placed on the chest. The shock can break elec -\ntrical chaos (ventricular fibrillation) and help to restart \nthe heart. Defibrillation within 3 minutes can, in some \ncases, increase the chance of survival up to 70%.\nEarly CPR  \n– to buy time\nEarly CPR increases the chance of sur -\nvival by 2-3 times compared to if no CPR \nis started before the ambulance arrives. \nIf a person suffers from a cardiac arrest, \nCPR should be started immediately. \nThrough CPR, a method consisting of \nchest compressions and rescue breaths, \nblood is oxyge

In [106]:
docs_inspect = retriever.get_relevant_documents("Claim")

In [105]:
docs_inspect = retriever.get_relevant_documents("FWD Critical Illness Plus")

In [80]:
[i.metadata for i in docs_inspect]

[{'Source': 'web', 'functions': 'claims'},
 {'Source': 'web', 'functions': 'claims'},
 {'Source': 'web', 'functions': 'disease'},
 {'Source': 'web', 'functions': 'claims'},
 {'Source': 'web', 'functions': 'disease'},
 {'Source': 'pdf', 'functions': 'disease'},
 {'Source': 'web', 'functions': 'disease'},
 {'Source': 'web', 'functions': 'disease'},
 {'Source': 'pdf', 'functions': 'disease'},
 {'Source': 'pdf', 'functions': 'disease'},
 {'Source': 'pdf', 'functions': 'disease'},
 {},
 {'Source': 'pdf', 'functions': 'claims'},
 {'Source': 'pdf', 'functions': 'disease'},
 {}]

In [64]:
# help(retriever.get_relevant_documents)

## Generator

The Generator prompts the LLM for an answer when the user asks a question. The retriever you created in the previous stage from the Chroma vector store will be used to pass relevant embeddings from the website data to the LLM to provide more context to the user's query.

You'll perform the following steps in this stage:

1. Chain together the following:
    * A prompt for extracting the relevant embeddings using the retriever.
    * A prompt for answering any question using LangChain.
    * An LLM model from Gemini for prompting.
    
2. Run the created chain with a question as input to prompt the model for an answer.


### Initialize Gemini

You must import `ChatGoogleGenerativeAI` from LangChain to initialize your model.
 In this example, you will use **gemini-pro**, as it supports text summarization. To know more about the text model, read Google AI's [language documentation](https://ai.google.dev/models/gemini).

You can configure the model parameters such as ***temperature*** or ***top_p***,  by passing the appropriate values when initializing the `ChatGoogleGenerativeAI` LLM.  To learn more about the parameters and their uses, read Google AI's [concepts guide](https://ai.google.dev/docs/concepts#model_parameters).

In [65]:
from langchain_google_genai import ChatGoogleGenerativeAI

# If there is no environment variable set for the API key, you can pass the API
# key to the parameter `google_api_key` of the `ChatGoogleGenerativeAI` function:
# `google_api_key="key"`.
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.1, top_p=0.85, google_api_key=google_api_key)

### Create prompt templates

You'll use LangChain's [PromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/) to generate prompts to the LLM for answering questions.

In the `llm_prompt`, the variable `question` will be replaced later by the input question, and the variable `context` will be replaced by the relevant text from the website retrieved from the Chroma vector store.

In [66]:
# Prompt template to query Gemini
llm_prompt_template = """You are an assistant for question-answering tasks.
Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Use five sentences maximum and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks.\nUse the following context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse five sentences maximum and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:"


### Create a stuff documents chain

LangChain provides [Chains](https://python.langchain.com/docs/modules/chains/) for chaining together LLMs with each other or other components for complex applications. You will create a **stuff documents chain** for this application. A stuff documents chain lets you combine all the relevant documents, insert them into the prompt, and pass that prompt to the LLM.

You can create a stuff documents chain using the [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language).

To learn more about different types of document chains, read LangChain's [chains guide](https://python.langchain.com/docs/modules/chains/document/).

The stuff documents chain for this application retrieves the relevant website data and passes it as the context to an LLM prompt along with the input question.

In [67]:
# Combine data from documents to readable string format.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create stuff documents chain using LCEL.
#
# This is called a chain because you are chaining together different elements
# with the LLM. In the following example, to create the stuff chain, you will
# combine the relevant context from the website data matching the question, the
# LLM model, and the output parser together like a chain using LCEL.
#
# The chain implements the following pipeline:
# 1. Extract the website data relevant to the question from the Chroma
#    vector store and save it to the variable `context`.
# 2. `RunnablePassthrough` option to provide `question` when invoking
#    the chain.
# 3. The `context` and `question` are then passed to the prompt where they
#    are populated in the respective variables.
# 4. This prompt is then passed to the LLM (`gemini-pro`).
# 5. Output from the LLM is passed through an output parser
#    to structure the model's response.
context = retriever | format_docs
rag_chain = (
    {"context": context, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

### Prompt the model

You can now query the LLM by passing any question to the `invoke()` function of the stuff documents chain you created previously.

In [69]:
response = rag_chain.invoke("provide an summary on cardiac arrest")
print("Context:", context| format_docs)  # Print the captured context
print("LLM Response:", response)

Context: first=VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f61486718d0>, search_kwargs={'k': 15}) middle=[RunnableLambda(format_docs)] last=RunnableLambda(format_docs)
LLM Response: Cardiac arrest is a life-threatening condition in which the heart suddenly stops beating. It's commonly caused by a heart attack or an inherited heart condition. Early recognition, CPR, defibrillation, and post-resuscitation care are crucial for improving the chances of survival.


In [70]:
rag_chain.invoke("What should i avoid doing if I have cardiac arrest")

"I'm sorry, but the context does not contain information about what to avoid doing while experiencing cardiac arrest. Therefore, I cannot answer your question."

In [71]:
rag_chain.invoke("how do i claim for heart disease for prudential", config={'functions':'claims'})

'I apologize, but the provided context does not contain information on how to claim for heart disease for Prudential. Therefore, I cannot answer your question.'

In [72]:
rag_chain.invoke("how do i claim for critical illness for FWD", config={'functions':'claims'})

'I apologize, but the provided context does not contain information on how to claim for critical illness for FWD. Therefore, I cannot answer your question.'

# Conclusion

That's it. You have successfully created an LLM application that answers questions using data from a website with the help of Gemini, LangChain, and Chroma.